# MotoPricePredictor: EDA

This notebook contains the exploratory data analysis (EDA) and preprocessing steps for the motorcycle price dataset.

The objective is to understand:
- The structure and quality of the dataset
- Distribution of prices and features
- Potential data quality issues
- Feature importance and predictive potential

This analysis will guide feature engineering and model selection in later stages.

In [ ]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = "../data/raw/MotorcycleData.csv"

In [7]:
df = pd.read_csv(data_path)
df.head().T

,0,1,2,3,4
id,0,1,2,3,4
Condition,Used,Used,Used,Used,Used
Condition_Desc,mint!!! very low miles,Perfect condition,NaN,CLEAN TITLE READY TO RIDE HOME,NaN
Price,"$11,412","$17,200","$3,872","$6,575","$10,000"
Location,"McHenry, Illinois, United States","Fort Recovery, Ohio, United States","Chicago, Illinois, United States","Green Bay, Wisconsin, United States","West Bend, Wisconsin, United States"
Model_Year,2013.0,2016.0,1970.0,2009.0,2012.0
Mileage,"16,000",60,"25,763","33,142","17,800"
Exterior_Color,Black,Black,Silver/Blue,Red,Blue
Make,Harley-Davidson,Harley-Davidson,BMW,Harley-Davidson,Harley-Davidson
Warranty,Unspecified,Vehicle has an existing warranty,Vehicle does NOT have an existing warranty,NaN,NO WARRANTY


In [8]:
df.shape

(7493, 23)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7493 entries, 0 to 7492
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              7493 non-null   int64  
 1   Condition       7493 non-null   object 
 2   Condition_Desc  1656 non-null   object 
 3   Price           7493 non-null   object 
 4   Location        7491 non-null   object 
 5   Model_Year      7489 non-null   float64
 6   Mileage         7468 non-null   object 
 7   Exterior_Color  6778 non-null   object 
 8   Make            7489 non-null   object 
 9   Warranty        5077 non-null   object 
 10  Model           7434 non-null   object 
 11  Sub_Model       2179 non-null   object 
 12  Type            6257 non-null   object 
 13  Vehicle_Title   0 non-null      float64
 14  OBO             7493 non-null   bool   
 15  Feedback_Perc   6785 non-null   object 
 16  Watch_Count     3278 non-null   object 
 17  N_Reviews       7493 non-null   i

## Data Preparation

- Is there any missing data?
- Update column names (remove spaces, lowercase, brackets)
- Identify column data types


### Missing Values

In [11]:
df.isnull().sum().sort_values(ascending=False)

Vehicle_Title     7493
Condition_Desc    5837
Sub_Model         5314
Bid_Count         5272
Watch_Count       4215
Warranty          2416
Type              1236
Exterior_Color     715
Feedback_Perc      708
Seller_Status      632
Model               59
Vehicle_Tile        54
Mileage             25
Model_Year           4
Make                 4
Location             2
Price                0
id                   0
Condition            0
OBO                  0
N_Reviews            0
Auction              0
Buy_Now              0
dtype: int64

*Missing Value Analysis*

The dataset contains a mixture of complete and partially missing fields.
There are several ways we can handle these missing values such as keeping them, imputing them or removing them all together.

Vehichle title seems to be completely empty and so can be removed along with the id which is redundant, as this is just inline with index values.

Condition_Desc can also be removed as we will not be using the freetext at this stage, though in the future, it could be possible to apply sentiment analysis to get a score for condition to be used in prediction. That is beyond the scope of this project at this point.

Numerical fields such as Mileage and Model_Year contained very few missing values and can be imputed using the median to preserve distributional properties.

The target variable (Price) contained no missing values.

We will first look at the data and the distribution of it before making anymore decisions on what fields to drop.

In [12]:
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
categorical_cols

['Condition',
 'Condition_Desc',
 'Price',
 'Location',
 'Mileage',
 'Exterior_Color',
 'Make',
 'Warranty',
 'Model',
 'Sub_Model',
 'Type',
 'Feedback_Perc',
 'Watch_Count',
 'Seller_Status',
 'Vehicle_Tile']

In [14]:
df[categorical_cols].nunique()

Condition            2
Condition_Desc    1184
Price             1748
Location          2026
Mileage           3984
Exterior_Color     869
Make                59
Warranty            69
Model             1748
Sub_Model         1207
Type               230
Feedback_Perc       63
Watch_Count         76
Seller_Status       29
Vehicle_Tile         4
dtype: int64

In [18]:
for category in categorical_cols:
    print(df[category].value_counts())
    print("="*50)

Condition
Used    6564
New      929
Name: count, dtype: int64
Condition_Desc
Excellent condition                                                                                        20
Excellent Condition                                                                                        12
Excellent                                                                                                  12
used item, normal wear and tear, Runs and Rides Great, CLEAN AND CLEAR TITLE, Tested and Inspected,        11
Like new                                                                                                   11
                                                                                                           ..
CLEAN TITLE   READY TO RIDE HOME                                                                            1
It&#039;s a &#039;72 in good shape                                                                          1
Overall, this bike is in good condition for